In [1]:
cd ../..

D:\Users\mrefoyo\Proyectos\Sub-SpaCE_plus


In [17]:
import numpy as np
import pandas as pd

from experiments.experiment_utils import local_data_loader

In [56]:
# models = ['cls_basic_train', ""]
# models = ['cls_basic_train', "inceptiontime_noscaling"]
models = ["inceptiontime_noscaling"]

"""univariate_datasets = [
    'Adiac', 'ArrowHead', 'Beef', 'BeetleFly', 'BirdChicken', 'Car', 'CBF',
    'ChlorineConcentration', 'CinCECGTorso', 'Coffee', 'Computers',
    'DiatomSizeReduction',
    'DistalPhalanxOutlineCorrect', 'DistalPhalanxOutlineAgeGroup',
    'DistalPhalanxTW', 'Earthquakes', 'ECG200', 'ECG5000', 'ECGFiveDays',
    'ElectricDevices', 'FaceAll', 'FaceFour', 'FacesUCR', 'FiftyWords', 'Fish',
    'FordA', 'FordB', 'GunPoint', 'Ham', 'HandOutlines', 'Haptics', 'Herring',
    'InlineSkate', 'InsectWingbeatSound', 'ItalyPowerDemand',
    'LargeKitchenAppliances', 'Lightning2', 'Lightning7', 'Mallat', 'Meat',
    'MedicalImages', 'MiddlePhalanxOutlineCorrect',
    'MiddlePhalanxOutlineAgeGroup', 'MiddlePhalanxTW', 'MoteStrain',
    'NonInvasiveFatalECGThorax1', 'NonInvasiveFatalECGThorax2', 'OliveOil',
    'OSULeaf', 'PhalangesOutlinesCorrect', 'Phoneme', 'Plane',
    'ProximalPhalanxOutlineCorrect', 'ProximalPhalanxOutlineAgeGroup',
    'ProximalPhalanxTW', 'RefrigerationDevices', 'ScreenType', 'ShapeletSim',
    'ShapesAll', 'SmallKitchenAppliances', 'SonyAIBORobotSurface1',
    'SonyAIBORobotSurface2', 'StarLightCurves', 'Strawberry', 'SwedishLeaf',
    'Symbols', 'SyntheticControl', 'ToeSegmentation1', 'ToeSegmentation2',
    'Trace', 'TwoLeadECG', 'TwoPatterns',
    'Wafer', 'Wine', 'WordSynonyms', 'Worms', 'WormsTwoClass', 'Yoga'
]"""
univariate_datasets = [
    'ECG200', 'Gunpoint', 'Coffee',
    'ItalyPowerDemand', 'ProximalPhalanxOutlineCorrect', 'Strawberry', 'FordA', 'HandOutlines',
    'Plane', 'TwoPatterns', 'FacesUCR', 'ECG5000', 'CinCECGTorso', 'NonInvasiveFatalECGThorax2', 'CBF',
]
# univariate_datasets = []

"""multivariate_datasets = [
    'BasicMotions', 'NATOPS', 'UWaveGestureLibrary',
    'ArticularyWordRecognition', 'AtrialFibrillation', 'CharacterTrajectories', 'Cricket',
    'DuckDuckGeese', 'EigenWorms', 'Epilepsy', 'EthanolConcentration', 'ERing',  'FaceDetection',
    'FingerMovements', 'HandMovementDirection', 'Handwriting', 'Heartbeat', 'InsectWingbeat', 'JapaneseVowels',
    'Libras', 'LSST', 'MotorImagery', 'PenDigits', 'PEMS-SF', 'Phoneme', 'RacketSports',
    'SelfRegulationSCP1', 'SelfRegulationSCP2', 'SpokenArabicDigits', 'StandWalkJump'
]"""
multivariate_datasets = []

datasets = univariate_datasets + multivariate_datasets

In [57]:
scaling = "none"

In [58]:
best_results_df = pd.DataFrame()
for dataset in datasets:
    for model in models:
        try:
            # Obtain results of dataset
            dataset_results_df = pd.read_excel(f'./experiments/models/{dataset}/{model}/all_combination_results.xlsx', index_col=0)
            dataset_results_df.insert(0, "Dataset", dataset)
            dataset_results_df["model"] = model
            if "model_type" not in dataset_results_df.columns:
                dataset_results_df["model_type"] = model

            if "scaling" in dataset_results_df.columns:
                dataset_results_df = dataset_results_df[dataset_results_df["scaling"] == scaling]
                
            dataset_results_df = dataset_results_df.rename(columns={"model": "Model", "model_type": "Model Type", "test_f1": "F1-score"})  
            selected_cols = ['Dataset', 'Model', 'Model Type', 'F1-score']
            if "test_acc" in dataset_results_df.columns:
                selected_cols += ['test_acc']
                
            best_dataset_results_df = dataset_results_df[selected_cols].iloc[0:1]

            
            # Obtain dataset information
            X_train, y_train, X_test, y_test = local_data_loader(str(dataset), "standard", data_path="./experiments/data")
            if X_train.shape[2] > 1:
                best_dataset_results_df.insert(1,'Archive', "UEA")
            else:
                best_dataset_results_df.insert(1,'Archive', "UCR")
            best_dataset_results_df.insert(1,'Classes', len(np.unique(y_train)))
            best_dataset_results_df.insert(1,'Channels', X_train.shape[2])
            best_dataset_results_df.insert(1, 'Length', X_train.shape[1])
            best_dataset_results_df.insert(1, 'Test size', X_test.shape[0])
            best_dataset_results_df.insert(1, 'Train size', X_train.shape[0])

            # Concat results
            best_results_df = pd.concat([best_results_df, best_dataset_results_df])
        except FileNotFoundError as msg:
            print(f'Problem with dataset {dataset}: {msg}')

# Filter and sort results df
best_results_df = best_results_df.sort_values(by='Dataset', ascending=False).reset_index(drop=True)


In [59]:
aux = best_results_df.sort_values(by=["Dataset", "Archive", "F1-score"], ascending=[True, True, False])
# aux = aux.set_index("Dataset").drop(["Archive", "Model Type"], axis=1)
aux = aux.drop(["Archive", "Model", "Model Type", "test_acc"], axis=1)
aux = aux[aux["F1-score"] > 0.8]
aux = aux.set_index("Dataset").sort_values(by="F1-score", ascending=False)
aux = aux.style.format(decimal='.', precision=3)
aux

,Train size,Test size,Length,Channels,Classes,F1-score
Dataset,,,,,,
Coffee,28,28,1,286,2,1.000
Gunpoint,50,150,1,150,2,1.000
Plane,105,105,1,144,7,1.000
TwoPatterns,1000,4000,1,128,4,1.000
CBF,30,900,1,128,3,0.999
Strawberry,613,370,1,235,2,0.981
FordA,3601,1320,1,500,2,0.959
ItalyPowerDemand,67,1029,1,24,2,0.958
FacesUCR,200,2050,1,131,14,0.951


In [60]:
print(aux.to_latex())

\begin{tabular}{lrrrrrr}
 & Train size & Test size & Length & Channels & Classes & F1-score \\
Dataset &  &  &  &  &  &  \\
Coffee & 28 & 28 & 1 & 286 & 2 & 1.000 \\
Gunpoint & 50 & 150 & 1 & 150 & 2 & 1.000 \\
Plane & 105 & 105 & 1 & 144 & 7 & 1.000 \\
TwoPatterns & 1000 & 4000 & 1 & 128 & 4 & 1.000 \\
CBF & 30 & 900 & 1 & 128 & 3 & 0.999 \\
Strawberry & 613 & 370 & 1 & 235 & 2 & 0.981 \\
FordA & 3601 & 1320 & 1 & 500 & 2 & 0.959 \\
ItalyPowerDemand & 67 & 1029 & 1 & 24 & 2 & 0.958 \\
FacesUCR & 200 & 2050 & 1 & 131 & 14 & 0.951 \\
HandOutlines & 1000 & 370 & 1 & 2709 & 2 & 0.948 \\
NonInvasiveFatalECGThorax2 & 1800 & 1965 & 1 & 750 & 42 & 0.948 \\
ProximalPhalanxOutlineCorrect & 600 & 291 & 1 & 80 & 2 & 0.913 \\
ECG5000 & 500 & 4500 & 1 & 140 & 5 & 0.901 \\
ECG200 & 100 & 100 & 1 & 96 & 2 & 0.876 \\
CinCECGTorso & 40 & 1380 & 1 & 1639 & 4 & 0.856 \\
\end{tabular}



In [28]:
aux = best_results_df.copy()
aux = aux[aux["F1-score"] > 0.8]
aux = aux[aux["Archive"] == "UEA"]
aux = aux.reset_index(drop=True).sort_values(by=["Classes", "Dataset"], ascending=[True, True])
aux

,Dataset,Train size,Test size,Length,Channels,Classes,Archive,Model,Model Type,F1-score,test_acc
1,SelfRegulationSCP1,268,293,6,896,2,UEA,inceptiontime_noscaling,inceptiontime_noscaling,0.837337,0.839590
8,BasicMotions,40,40,6,100,4,UEA,inceptiontime_noscaling,inceptiontime_noscaling,1.000000,1.000000
6,Epilepsy,137,138,3,206,4,UEA,inceptiontime_noscaling,inceptiontime_noscaling,0.970925,0.971014
2,RacketSports,151,152,6,30,4,UEA,inceptiontime_noscaling,inceptiontime_noscaling,0.874927,0.875000
5,NATOPS,180,180,24,51,6,UEA,inceptiontime_noscaling,inceptiontime_noscaling,0.933268,0.933333
4,PEMS-SF,267,173,963,144,7,UEA,inceptiontime_noscaling,inceptiontime_noscaling,0.813037,0.815029
0,UWaveGestureLibrary,120,320,3,315,8,UEA,inceptiontime_noscaling,inceptiontime_noscaling,0.883169,0.884375
3,PenDigits,7494,3498,2,8,10,UEA,inceptiontime_noscaling,inceptiontime_noscaling,0.988271,0.988279
7,Cricket,108,72,6,1197,12,UEA,inceptiontime_noscaling,inceptiontime_noscaling,0.869943,0.902778
9,ArticularyWordRecognition,275,300,9,144,25,UEA,inceptiontime_noscaling,inceptiontime_noscaling,0.872147,0.900000


In [8]:
aux["Dataset"].values

array(['Coffee', 'ECG200', 'ECGFiveDays', 'FordA', 'GunPoint',
       'HandOutlines', 'ItalyPowerDemand', 'MiddlePhalanxOutlineCorrect',
       'MoteStrain', 'PhalangesOutlinesCorrect',
       'ProximalPhalanxOutlineCorrect', 'SonyAIBORobotSurface2',
       'Strawberry', 'Wafer', 'CBF', 'ChlorineConcentration',
       'ProximalPhalanxOutlineAgeGroup', 'StarLightCurves',
       'CinCECGTorso', 'FaceFour', 'Trace', 'TwoPatterns', 'ECG5000',
       'SyntheticControl', 'Fish', 'Plane', 'FaceAll', 'FacesUCR',
       'SwedishLeaf', 'NonInvasiveFatalECGThorax1',
       'NonInvasiveFatalECGThorax2'], dtype=object)

In [29]:
def create_latex_table(results_df_orig, ordered_models, higher_is_better=True):
    results_df = results_df_orig.copy()
    # Order methods and datasets
    ordered_models = [model for model in ordered_models if model in results_df.columns]
    results_df = results_df[ordered_models]
    model_names = results_df.columns

    # Create table header
    table_header_fixed = """
\\begin{table}[ht]
\\centering
\\begin{tabular}{l|""" + "".join(["c"] * len(model_names)) + "}\n\\toprule\n"

    table_columns = "Dataset & " + " & ".join(model_names) + " \\\\\n\midrule\n"

    table_rows = []
    for dataset, row in results_df.iterrows():
        row_data = [dataset]
        # Extract numeric values and determine the best and second-best
        
        numeric_values = row[model_names].dropna()
        if higher_is_better:
            best_value = numeric_values.max()
            second_best_value = numeric_values.nlargest(2).iloc[-1]
        else:
            best_value = numeric_values.min()
            second_best_value = numeric_values.nsmallest(2).iloc[-1]
            
        for model in model_names:
            value = row[model]
            if pd.isna(value):
                formatted_value = "-"
            else:
                formatted_value = str(round(value, 2))
                if value == best_value:
                    formatted_value = f"\\textbf{{{formatted_value}}}"
                elif value == second_best_value:
                    formatted_value = f"\\underline{{{formatted_value}}}"
            row_data.append(formatted_value)
        table_rows.append(" & ".join(row_data) + " \\\\")

    table_footer = """
\\bottomrule
\\end{tabular}
\\end{table}
"""
    latex_table = table_header_fixed + table_columns + "\n".join(table_rows) + table_footer
    print(latex_table)

# Classifier comparison table

In [31]:
all_results_df = pd.DataFrame()
for dataset in datasets:
    for model in models:
        try:
            # Obtain results of dataset
            dataset_results_df = pd.read_excel(f'./experiments/models/{dataset}/{model}/all_combination_results.xlsx', index_col=0)
            dataset_results_df.insert(0, "Dataset", dataset)
            dataset_results_df = dataset_results_df.rename(columns={"model_type": "Model Type", "test_f1": "F1-score"})

            # Obtain dataset information
            X_train, y_train, X_test, y_test = local_data_loader(str(dataset), scaling, data_path="./experiments/data")
            if X_train.shape[2] > 1:
                dataset_results_df.insert(1,'Archive', "UEA")
            else:
                dataset_results_df.insert(1,'Archive', "UCR")

            # Concat results
            all_results_df = pd.concat([all_results_df, dataset_results_df])
        except FileNotFoundError as msg:
            print(f'Problem with dataset {dataset}: {msg}')

Problem with dataset AtrialFibrillation: [Errno 2] No such file or directory: './experiments/models/AtrialFibrillation/inceptiontime_noscaling/all_combination_results.xlsx'
Problem with dataset CharacterTrajectories: [Errno 2] No such file or directory: './experiments/models/CharacterTrajectories/inceptiontime_noscaling/all_combination_results.xlsx'
Problem with dataset DuckDuckGeese: [Errno 2] No such file or directory: './experiments/models/DuckDuckGeese/inceptiontime_noscaling/all_combination_results.xlsx'
Problem with dataset EigenWorms: [Errno 2] No such file or directory: './experiments/models/EigenWorms/inceptiontime_noscaling/all_combination_results.xlsx'
Problem with dataset ERing: [Errno 2] No such file or directory: './experiments/models/ERing/inceptiontime_noscaling/all_combination_results.xlsx'
Problem with dataset InsectWingbeat: [Errno 2] No such file or directory: './experiments/models/InsectWingbeat/inceptiontime_noscaling/all_combination_results.xlsx'
Problem with dat

In [32]:
all_results_univariate = all_results_df[all_results_df["Archive"] == "UCR"]
all_results_univariate = all_results_univariate[["Dataset", "Model Type", "F1-score"]].pivot_table(index="Dataset", columns="Model Type", values="F1-score")
all_results_univariate["Best Score"] = all_results_univariate.max(axis=1)
all_results_univariate = all_results_univariate.sort_values(by="Best Score", ascending=False)
# all_results_univariate = all_results_univariate[all_results_univariate["Best Score"] > 0.8]
all_results_univariate.index.name = None
all_results_univariate.columns.name = None

all_results_multivariate = all_results_df[all_results_df["Archive"] == "UEA"]
all_results_multivariate = all_results_multivariate[["Dataset", "Model Type", "F1-score"]].pivot_table(index="Dataset", columns="Model Type", values="F1-score")
all_results_multivariate["Best Score"] = all_results_multivariate.max(axis=1)
all_results_multivariate = all_results_multivariate.sort_values(by="Best Score", ascending=False)
# all_results_multivariate = all_results_multivariate[all_results_multivariate["Best Score"] > 0.8]
all_results_multivariate.index.name = None
all_results_multivariate.columns.name = None

KeyError: "['Model Type'] not in index"

In [12]:
create_latex_table(all_results_univariate, ["cnn-simple", "cnn-intermediate", "cnn-complex"], higher_is_better=True)


\begin{table}[ht]
\centering
\begin{tabular}{l|ccc}
\toprule
Dataset & cnn-simple & cnn-intermediate & cnn-complex \\
\midrule
TwoPatterns & 0.99 & \underline{1.0} & \textbf{1.0} \\
Wafer & 0.99 & \underline{0.99} & \textbf{0.99} \\
SyntheticControl & \underline{0.98} & \textbf{0.98} & 0.97 \\
Strawberry & 0.97 & \underline{0.98} & \textbf{0.98} \\
Plane & \underline{0.33} & 0.05 & \textbf{0.98} \\
StarLightCurves & \underline{0.96} & 0.96 & \textbf{0.97} \\
ItalyPowerDemand & \textbf{0.96} & \underline{0.96} & 0.33 \\
CinCECGTorso & \textbf{0.95} & \underline{0.77} & 0.71 \\
GunPoint & 0.33 & \underline{0.41} & \textbf{0.95} \\
Coffee & 0.29 & \underline{0.44} & \textbf{0.93} \\
HandOutlines & \underline{0.92} & \textbf{0.92} & 0.89 \\
FordA & \underline{0.9} & 0.89 & \textbf{0.92} \\
NonInvasiveFatalECGThorax2 & 0.9 & \underline{0.91} & \textbf{0.91} \\
SwedishLeaf & 0.87 & \underline{0.88} & \textbf{0.91} \\
ECGFiveDays & \textbf{0.91} & 0.62 & \underline{0.88} \\
Trace & \underlin

In [13]:
create_latex_table(all_results_multivariate, ["cnn-simple", "cnn-intermediate", "cnn-complex"], higher_is_better=True)


\begin{table}[ht]
\centering
\begin{tabular}{l|ccc}
\toprule
Dataset & cnn-simple & cnn-intermediate & cnn-complex \\
\midrule
PenDigits & \underline{0.98} & \textbf{0.98} & 0.98 \\
SelfRegulationSCP1 & \underline{0.87} & \textbf{0.9} & 0.85 \\
NATOPS & \textbf{0.89} & \underline{0.87} & 0.84 \\
Epilepsy & \underline{0.76} & \textbf{0.88} & 0.24 \\
PEMS-SF & \underline{0.86} & \textbf{0.87} & 0.8 \\
ArticularyWordRecognition & \textbf{0.86} & \underline{0.85} & 0.85 \\
UWaveGestureLibrary & \textbf{0.85} & 0.81 & \underline{0.82} \\
Cricket & \textbf{0.84} & \underline{0.8} & 0.51 \\
RacketSports & \textbf{0.84} & \underline{0.71} & 0.69 \\
BasicMotions & \textbf{0.84} & \underline{0.71} & 0.67 \\
FaceDetection & \textbf{0.66} & \underline{0.65} & 0.64 \\
Heartbeat & \underline{0.64} & 0.63 & \textbf{0.65} \\
FingerMovements & \textbf{0.55} & \underline{0.55} & \underline{0.55} \\
HandMovementDirection & \textbf{0.53} & \underline{0.49} & 0.47 \\
SelfRegulationSCP2 & 0.45 & \underline

# Autoencoder comparison table

In [150]:
models = ["ae_basic_train"]

rename_dict = {
    'ae-cnn-shallow': "Shallow",
    'ae-cnn-simple': "Simple",
    'ae-cnn-intermediate': "Interm.",
    'ae-cnn-complex': "Complex",
    "0.0625": "6,25\%",
    "0.125": "12,5\%"
}

In [159]:
ae_univariate_datasets = [
    'ECG200', 'Gunpoint', 'Coffee',
    'ItalyPowerDemand', 'ProximalPhalanxOutlineCorrect', 'Strawberry', 'FordA', 'HandOutlines',
    'Plane', 'TwoPatterns', 'FacesUCR', 'ECG5000', 'CinCECGTorso', 'NonInvasiveFatalECGThorax2', 'CBF',
]

ae_multivariate_datasets = [
    'BasicMotions', 'NATOPS', 'UWaveGestureLibrary',
    'ArticularyWordRecognition', 'Cricket',
    'Epilepsy', 'PenDigits',
    'PEMS-SF', 'RacketSports', 'SelfRegulationSCP1'
]

ae_datasets = ae_univariate_datasets + ae_multivariate_datasets

In [160]:
all_results_df = pd.DataFrame()
for dataset in ae_datasets:
    for model in models:
        try:
            # Obtain results of dataset
            dataset_results_df = pd.read_excel(f'./experiments/models/{dataset}/{model}/all_combination_results.xlsx', index_col=0)
            dataset_results_df.insert(0, "Dataset", dataset)
            dataset_results_df = dataset_results_df.rename(columns={"model_type": "Model Type", "test_mae": "MAE"})
            dataset_results_df["compression_rate"] = dataset_results_df["compression_rate"].astype(str)
            dataset_results_df = dataset_results_df.replace(rename_dict)

            # Obtain dataset information
            X_train, y_train, X_test, y_test = local_data_loader(str(dataset), False, data_path="./experiments/data")
            if X_train.shape[2] > 1:
                dataset_results_df.insert(1,'Archive', "UEA")
            else:
                dataset_results_df.insert(1,'Archive', "UCR")

            # Concat results
            all_results_df = pd.concat([all_results_df, dataset_results_df])
        except FileNotFoundError as msg:
            print(f'Problem with dataset {dataset}: {msg}')

In [161]:
all_results_univariate = all_results_df[all_results_df["Archive"] == "UCR"]
all_results_univariate["Model Type"] = all_results_univariate["Model Type"] + '(' + all_results_univariate["compression_rate"] + ')'
all_results_univariate = all_results_univariate[["Dataset", "Model Type", "MAE"]].pivot_table(index="Dataset", columns="Model Type", values="MAE")
all_results_univariate["Best Score"] = all_results_univariate.min(axis=1)
all_results_univariate = all_results_univariate.sort_values(by="Best Score", ascending=True)
# all_results_univariate = all_results_univariate[all_results_univariate["Best Score"] > 0.8]
all_results_univariate.index.name = None
all_results_univariate.columns.name = None

all_results_multivariate = all_results_df[all_results_df["Archive"] == "UEA"]
all_results_multivariate["Model Type"] = all_results_multivariate["Model Type"] + '(' + all_results_multivariate["compression_rate"] + ')'
all_results_multivariate = all_results_multivariate[["Dataset", "Model Type", "MAE"]].pivot_table(index="Dataset", columns="Model Type", values="MAE")
all_results_multivariate["Best Score"] = all_results_multivariate.min(axis=1)
all_results_multivariate = all_results_multivariate.sort_values(by="Best Score", ascending=True)
# all_results_multivariate = all_results_multivariate[all_results_multivariate["Best Score"] > 0.8]
all_results_multivariate.index.name = None
all_results_multivariate.columns.name = None

C:\Users\mrefoyo\AppData\Local\Temp\ipykernel_10988\666821464.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_results_univariate["Model Type"] = all_results_univariate["Model Type"] + '(' + all_results_univariate["compression_rate"] + ')'
C:\Users\mrefoyo\AppData\Local\Temp\ipykernel_10988\666821464.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_results_multivariate["Model Type"] = all_results_multivariate["Model Type"] + '(' + all_results_multivariate["compression_rate"] + ')'


In [162]:
all_results_multivariate

,Complex(0.25),"Complex(12,5\%)","Complex(6,25\%)",Interm.(0.25),"Interm.(12,5\%)","Interm.(6,25\%)",Shallow(0.25),"Shallow(12,5\%)","Shallow(6,25\%)",Simple(0.25),"Simple(12,5\%)","Simple(6,25\%)",Best Score
PEMS-SF,NaN,0.017146,0.016774,NaN,0.017007,0.016509,NaN,0.017523,0.015391,NaN,0.016231,0.017485,0.015391
UWaveGestureLibrary,0.084881,0.091011,0.109035,0.067678,0.070607,0.388067,0.516513,NaN,NaN,0.063299,0.384162,NaN,0.063299
NATOPS,0.138464,0.139117,0.139285,0.120989,0.134623,0.125641,0.096402,0.104329,0.136016,0.107795,0.106435,0.111035,0.096402
Cricket,NaN,0.174872,0.191363,NaN,0.210031,0.298487,NaN,0.560322,NaN,NaN,0.317323,0.494582,0.174872
ArticularyWordRecognition,NaN,0.275677,0.277416,NaN,0.225305,0.334598,NaN,0.528095,0.560541,NaN,0.332640,0.431921,0.225305
Epilepsy,0.315425,0.338835,0.364343,0.258223,0.310258,0.429648,0.425127,NaN,NaN,0.256223,0.421535,NaN,0.256223
BasicMotions,1.800046,1.787157,1.905056,1.606032,1.576541,1.627191,1.167951,1.421965,NaN,1.338827,1.341668,1.587692,1.167951
RacketSports,NaN,2.758823,2.812440,NaN,2.732919,2.760923,NaN,2.757638,NaN,NaN,2.565719,2.766061,2.565719
PenDigits,NaN,4.043865,6.293295,NaN,4.246509,6.567308,NaN,NaN,NaN,NaN,5.028624,NaN,4.043865
SelfRegulationSCP1,NaN,5.938527,6.047909,NaN,7.479413,6.063556,NaN,9.931918,NaN,NaN,5.156580,4.957625,4.957625


In [164]:
ordered_models = [
    "Shallow(6,25\%)", "Shallow(12,5\%)", 
    "Simple(6,25\%)", "Simple(12,5\%)", 
    "Interm.(6,25\%)", "Interm.(12,5\%)", 
    "Complex(6,25\%)", "Complex(12,5\%)", 
]
                  

In [165]:
all_results_univariate.dtypes

Complex(12,5\%)    float64
Complex(6,25\%)    float64
Interm.(12,5\%)    float64
Best Score         float64
dtype: object

In [166]:
create_latex_table(all_results_univariate, ordered_models, higher_is_better=False)


\begin{table}[ht]
\centering
\begin{tabular}{l|ccc}
\toprule
Dataset & Interm.(12,5\%) & Complex(6,25\%) & Complex(12,5\%) \\
\midrule
HandOutlines & \textbf{0.0} & \underline{0.0} & 0.0 \\
Strawberry & \underline{0.02} & 0.02 & \textbf{0.01} \\
NonInvasiveFatalECGThorax2 & \underline{0.02} & 0.03 & \textbf{0.02} \\
ProximalPhalanxOutlineCorrect & \underline{0.02} & 0.02 & \textbf{0.02} \\
Plane & \underline{0.09} & 0.1 & \textbf{0.08} \\
ECG5000 & \underline{0.08} & 0.1 & \textbf{0.08} \\
CinCECGTorso & \textbf{0.11} & 0.16 & \underline{0.15} \\
FordA & \textbf{0.16} & 0.27 & \underline{0.16} \\
ECG200 & \textbf{0.17} & 0.2 & \underline{0.18} \\
ItalyPowerDemand & 0.26 & \textbf{0.19} & \underline{0.21} \\
TwoPatterns & \textbf{0.21} & 0.29 & \underline{0.22} \\
Gunpoint & 0.93 & \underline{0.29} & \textbf{0.28} \\
FacesUCR & \underline{0.44} & 0.52 & \textbf{0.41} \\
Coffee & \textbf{0.63} & \underline{0.73} & 0.82 \\
CBF & \textbf{0.68} & 0.75 & \underline{0.75} \\
\bottomrule
\end

In [167]:
create_latex_table(all_results_multivariate, ordered_models, higher_is_better=False)


\begin{table}[ht]
\centering
\begin{tabular}{l|cccccccc}
\toprule
Dataset & Shallow(6,25\%) & Shallow(12,5\%) & Simple(6,25\%) & Simple(12,5\%) & Interm.(6,25\%) & Interm.(12,5\%) & Complex(6,25\%) & Complex(12,5\%) \\
\midrule
PEMS-SF & \textbf{0.02} & 0.02 & 0.02 & \underline{0.02} & 0.02 & 0.02 & 0.02 & 0.02 \\
UWaveGestureLibrary & - & - & - & 0.38 & 0.39 & \textbf{0.07} & 0.11 & \underline{0.09} \\
NATOPS & 0.14 & \textbf{0.1} & 0.11 & \underline{0.11} & 0.13 & 0.13 & 0.14 & 0.14 \\
Cricket & - & 0.56 & 0.49 & 0.32 & 0.3 & 0.21 & \underline{0.19} & \textbf{0.17} \\
ArticularyWordRecognition & 0.56 & 0.53 & 0.43 & 0.33 & 0.33 & \textbf{0.23} & 0.28 & \underline{0.28} \\
Epilepsy & - & - & - & 0.42 & 0.43 & \textbf{0.31} & 0.36 & \underline{0.34} \\
BasicMotions & - & \underline{1.42} & 1.59 & \textbf{1.34} & 1.63 & 1.58 & 1.91 & 1.79 \\
RacketSports & - & 2.76 & 2.77 & \textbf{2.57} & 2.76 & \underline{2.73} & 2.81 & 2.76 \\
PenDigits & - & - & - & 5.03 & 6.57 & \underline{4.25} &

# Isolation Forest Comparison table

# LOF Comparison table